# Tutorial

In this notebook, we are going to have a good tour of main features, usages and how-to of the package.

## 1. Install

As requested we need to install the package from pypi. Depends of your internet connection, it could take 1 or 2 minutes.

In [2]:
!pip install legal_doc_processing==2.2.3

## 2.Check package integriity and download third parties objects

As we will use major NLP Librairies,we need to download data collections and mandory web assets such as NLTK stop words, Scapy or Transformers models or tensorflow/pyTorch assets. It can take 1/2 minutes (depends of your web connection)

In [ ]:

There is 3 main modules in legal-doc-processing : 
- legal_doc for LegalDoc objects ie order, complaint, etc etc official documents
- press_release for PressRelease objects for legal press release related to each case
- decision for both LegalDoc and PressRelease documents. The Decision object is able to read both, make prediction from both documents and merge/clean prediction of both documents. If just one of them, the object will apply basic method based on LegalDoc or PressRelease object

so you can :
```
from legal_doc_processing import *  # import all
from legal_doc_processing import legal_doc # import legal document module
from legal_doc_processing import press_release # import press release module
from legal_doc_processing import decision # import decision module
```
<br>

### Instanciation
youn can init an object in 2 ways.
* with text directly
```
from legal_doc_processing import ld
doc = ld.LegalDoc("this is a document")
# or better
doc = ld.from_text("this is a document")
```
* with path to a file
```
doc = ld.from_file("this/is/my/file.txt")
```

for press release, same pattern: 
```
from legal_doc_processing import pr
press = ld.PressRelease("this is a press release")
press = ld.from_text("this is a press release")
```
* with path to a file
```
press = ld.from_file("this/is/my/file.txt")
```

once instanciated, you can print : 
```
print(doc)
```
<br>

all interessing features are in feature_dict attribute
```
print(doc.feature_dict)
```
<br>

### Predictions

you can now make predictions : 
```
defendant = doc.predict("defendant")
print(defendant)
case = doc.predict("case")
print(case)
```
<br>

predict method return a something but alson work on the object itself : 
```
print(doc)
print(doc.feature_dict)
```
<br>

of course most easy is to predict all : 
```
features = doc.predict("all")
print(features)
print(doc)
```
<br>

you can acces to final predictions or detailed prediction: 
```
print(doc.feature_dict) # just the results
print(doc._feature_dict) # results + score for each result
```

<br>
<br>